In [ ]:
!pip install pyreadstat
import pandas as pd
import pyreadstat
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

data=pd.read_stata("Anxiety_data2.dta")
data=data[data['mental'].notna()]
data.isnull().sum()
data[["husband_edu","tv","radio","inte","newspaper"]]=data[["husband_edu","tv","radio","inte","newspaper"]].astype('category')

features=['country1', 'v012', 'edu','occupation_group2', 'mstatus', 'wind1', 'v730', 'husband_edu2',
          'husband_works_currently2', 'husband_occupation2', 'v632', 'v636', 'fper', 'v216', 'hhsex1', 'v218', 'v136', 'media1']
#features=['country1', 'age_group', 'v106','occupation_group', 'mstatus', 'wind1', 'husband_age_group', 'husband_edu',
          #'husband_works_currently2', 'husband_occupation', 'v632', 'v636', 'fper', 'v216', 'hhsex1', 'numofchild', 'media1']
#features = ['v106', 'v213', 'v216', 'v632', 'v636', 'v714', 'v811', 'country1', 'age_group', 'occupation_group', 'mstatus', 'area1', 'wind1', 'husband_age_group', 'husband_edu', 'husband_works_currently2', 'husband_occupation', 'fper', 'csex1', 'hhmembers', 'hhsex1', 'numofchild', 'tv', 'radio', 'inte', 'media1', 'newspaper', 'bmi4a']


In [ ]:
data[features].nunique()

,0
country1,3
v012,35
edu,3
occupation_group2,3
mstatus,2
wind1,3
v730,72
husband_edu2,5
husband_works_currently2,2
husband_occupation2,3


In [ ]:
target = 'mental'

# Separate features and target
X = data[features]
y = data[target]

# Encode categorical features and target variable
encoders = {}
for col in X.columns:
    if X[col].dtype == 'object' or X[col].dtype.name == 'category':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        encoders[col] = le

le_target = LabelEncoder()
y = le_target.fit_transform(y)

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)


X_imputed = imputer.fit_transform(X)

X = pd.DataFrame(X_imputed, columns=X.columns)

<ipython-input-5-384257b7568c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-5-384257b7568c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])
<ipython-input-5-384257b7568c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [ ]:
# Split the data into training and testing sets

!pip install pyreadstat imbalanced-learn

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Initialize models
logreg_model = LogisticRegression(random_state=42)
dt_model = DecisionTreeClassifier(random_state=42)
rf_model = RandomForestClassifier(random_state=42)
svm_model = SVC(probability=True, random_state=42)
knn_model = KNeighborsClassifier()
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Train individual models
logreg_model.fit(X_train_smote, y_train_smote)
dt_model.fit(X_train_smote, y_train_smote)
rf_model.fit(X_train_smote, y_train_smote)
svm_model.fit(X_train_smote, y_train_smote)
knn_model.fit(X_train_smote, y_train_smote)
xgb_model.fit(X_train_smote, y_train_smote)

base_models=[('lg', logreg_model),('dt', dt_model), ('rf', rf_model), ('svm', svm_model), ('knn', knn_model), ('xgb', xgb_model)]

# Ensemble model using VotingClassifier (hard voting)
ensemble_model = VotingClassifier(estimators=base_models, voting='hard')
ensemble_model.fit(X_train, y_train)

#Ensemble model using Stacking
stacking_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(max_iter=100))
stacking_model.fit(X_train, y_train)

# Make predictions
lg_pred = logreg_model.predict(X_test)
dt_pred = dt_model.predict(X_test)
rf_pred = rf_model.predict(X_test)
svm_pred = svm_model.predict(X_test)
knn_pred = knn_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)
ensemble_pred = ensemble_model.predict(X_test)
stack_pred = stacking_model.predict(X_test)

# Evaluate models
print("Logistic regression", accuracy_score(y_test, lg_pred))
print("Decision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_pred))
print("SVM Accuracy:", accuracy_score(y_test, svm_pred))
print("KNN Accuracy:", accuracy_score(y_test, knn_pred))
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_pred))
print("Ensemble Accuracy:", accuracy_score(y_test, ensemble_pred))
print("Stacking Accuracy:", accuracy_score(y_test, stack_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:02:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-

Logistic regression 0.5895605072017728
Decision Tree Accuracy: 0.6556690877754524
Random Forest Accuracy: 0.7568632278714761
SVM Accuracy: 0.5818047519389388
KNN Accuracy: 0.5467191924165948
XGBoost Accuracy: 0.7680659854733473
Ensemble Accuracy: 0.7742213467930568
Stacking Accuracy: 0.7732364889819032


In [ ]:
from sklearn.metrics import classification_report

def evaluate_and_print_report(model, X, y, model_name):
    predictions = model.predict(X)
    print(f"\n{model_name} Classification Report:")
    print(classification_report(y, predictions))
    print(f"{model_name} Accuracy: ", accuracy_score(y, predictions))

evaluate_and_print_report(logreg_model, X_train_smote, y_train_smote, "Logistic Train")
evaluate_and_print_report(dt_model, X_train_smote, y_train_smote, "Decision Tree Train")
evaluate_and_print_report(rf_model, X_train_smote, y_train_smote, "Random Forest Train")
evaluate_and_print_report(svm_model, X_train_smote, y_train_smote, "SVM Train")
evaluate_and_print_report(knn_model, X_train_smote, y_train_smote, "KNN Train")
evaluate_and_print_report(xgb_model, X_train_smote, y_train_smote, "XGBoost Train")
evaluate_and_print_report(ensemble_model, X_train_smote, y_train_smote, "Ensemble Train")
evaluate_and_print_report(stacking_model, X_train, y_train, "stacking Train")

evaluate_and_print_report(logreg_model, X_test, y_test, "Logistic Test")
evaluate_and_print_report(dt_model, X_test, y_test, "Decision Tree Test")
evaluate_and_print_report(rf_model, X_test, y_test, "Random Forest Test")
evaluate_and_print_report(svm_model, X_test, y_test, "SVM Test")
evaluate_and_print_report(knn_model, X_test, y_test, "KNN Test")
evaluate_and_print_report(xgb_model, X_test, y_test, "XGBoost Test")
evaluate_and_print_report(ensemble_model, X_test, y_test, "Ensemble Test")
evaluate_and_print_report(stacking_model, X_test, y_test, "stacking Test")


Logistic Train Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.61      0.58     25415
           1       0.57      0.52      0.54     25415

    accuracy                           0.56     50830
   macro avg       0.56      0.56      0.56     50830
weighted avg       0.56      0.56      0.56     50830

Logistic Train Accuracy:  0.5637418847137518

Decision Tree Train Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     25415
           1       1.00      0.99      0.99     25415

    accuracy                           0.99     50830
   macro avg       0.99      0.99      0.99     50830
weighted avg       0.99      0.99      0.99     50830

Decision Tree Train Accuracy:  0.9938618925831202

Random Forest Train Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     25415
           1   

In [ ]:
#cross validation score of each model

from sklearn.model_selection import cross_val_score

def evaluate_with_cross_validation(model, X, y, cv=5, scoring='accuracy'):
    scores = cross_val_score(model, X, y, cv=cv, scoring=scoring)
    return scores

# Evaluate models with cross-validation
models = {
    'Logistic Regression': logreg_model,
    'Decision Tree': dt_model,
    'Random Forest': rf_model,
    'SVM': svm_model,
    'KNN': knn_model,
    'XGBoost': xgb_model,
    'Ensemble': ensemble_model,
    #'Stacking': stacking_model # Uncomment if you use stacking model
}

for model_name, model in models.items():
    cv_scores = evaluate_with_cross_validation(model, X_train, y_train)
    print(f"{model_name} Cross-Validation Scores: {cv_scores}")
    print(f"{model_name} Mean CV Accuracy: {cv_scores.mean()}")
    print(f"{model_name} Std CV Accuracy: {cv_scores.std()}")
    print("-" * 30)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression Cross-Validation Scores: [0.78224069 0.78224069 0.7820868  0.7820868  0.78236109]
Logistic Regression Mean CV Accuracy: 0.7822032120987913
Logistic Regression Std CV Accuracy: 0.00010472812001500021
------------------------------
Decision Tree Cross-Validation Scores: [0.65835642 0.66312712 0.66882118 0.66743613 0.66230568]
Decision Tree Mean CV Accuracy: 0.6640093045761633
Decision Tree Std CV Accuracy: 0.0037558752416413507
------------------------------
Random Forest Cross-Validation Scores: [0.76977532 0.77208372 0.77162204 0.77100646 0.76973988]
Random Forest Mean CV Accuracy: 0.7708454829141257
Random Forest Std CV Accuracy: 0.0009518187237152879
------------------------------


In [ ]:
# prompt: perform Auroc for all models

from sklearn.metrics import roc_auc_score

# Assuming you have already trained your models and have X_test and y_test
#ann model
models = {
    'Logistic Regression': logreg_model,
    'Decision Tree': dt_model,
    'Random Forest': rf_model,
    'SVM': svm_model,
    'KNN': knn_model,
    'XGBoost': xgb_model,
    'Ensemble': ensemble_model,
    'Stacking': stacking_model
}

for model_name, model in models.items():
    try:
        # Predict probabilities for the positive class
        y_prob = model.predict_proba(X_test)[:, 1]  # Assuming binary classification
        auroc = roc_auc_score(y_test, y_prob)
        print(f"AUROC for {model_name}: {auroc}")
    except AttributeError:
        print(f"AUROC calculation not supported for {model_name} (predict_proba not available)")

AUROC for Logistic Regression: 0.5848186578591991
AUROC for Decision Tree: 0.522559996021808
AUROC for Random Forest: 0.5870364219420161
AUROC for SVM: 0.5846600326502833
AUROC for KNN: 0.5400280881841806
AUROC for XGBoost: 0.601749104993428
AUROC calculation not supported for Ensemble (predict_proba not available)
AUROC for Stacking: 0.6055521248847346
